# SQL 쿼리 생성을 위해 베드록 모델 호출하기

> *오류가 발생하면 이 노트북에서 사용하는 베드록 모델에 대한 허용 목록에 추가해야 할 수 있습니다*

> *이 노트북은 SageMaker Studio의 **`Data Science 3.0`** 커널에서 잘 작동해야 합니다*

## 소개

이 노트북에서는 LLM을 활용해 판매 데이터를 분석하기 위한 SQL 쿼리를 생성하는 방법을 보여드리겠습니다.

Boto3 API를 사용하여 베드록의 Claude V2 모델을 활용할 것입니다.

이 예시에서 사용된 프롬프트는 프롬프트 외에 다른 예시 텍스트를 제공하지 않기 때문에 제로샷 프롬프트라고 합니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다.*

#### 컨텍스트
아마존 베드록의 SQL 코드 생성 기능을 보여드리기 위해, Boto3 클라이언트를 활용해 아마존 베드록 API와 통신하는 방법을 살펴보겠습니다. 사용 가능한 다양한 구성과 함께 간단한 입력으로 원하는 출력을 얻을 수 있는 방법을 보여드리겠습니다.

#### 패턴
아마존 베드록 API에 작업, 지시사항, 모델 입력으로 구성된 입력을 제공하여 추가적인 예시 없이 출력을 생성하도록 할 것입니다. 이를 통해 강력한 LLM이 주어진 작업을 쉽게 이해하고 설득력 있는 출력을 생성하는 방법을 보여드리고자 합니다.

![](./images/sql-query-generation.png)

#### 활용 사례
판매 데이터를 분석하기 위한 SQL 쿼리를 생성하는 활용 사례를 살펴보겠습니다. 트렌드, 인기 제품, 평균 판매량에 초점을 맞추겠습니다.

#### 페르소나
마야는 AnyCompany의 비즈니스 분석가로, 주로 판매 및 재고 데이터에 집중하고 있습니다. 그녀는 스프레드시트 분석에서 데이터 기반 분석으로 전환하고 있으며, SQL을 활용해 특정 데이터 포인트를 효과적으로 가져오고자 합니다. 그녀는 LLM을 활용해 분석에 필요한 SQL 쿼리를 생성하고자 합니다.

#### 구현
이 활용 사례를 구현하기 위해, 이 노트북에서는 SQL 쿼리를 생성하는 방법을 보여드리겠습니다. Boto3 클라이언트와 아마존 베드록 API를 활용해 Anthropic의 Claude v2 모델을 사용할 것입니다.

## 설정

In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.309

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/45/78/e95de33d8850429f6740763eb2d5350c02fd39e916f2ddb69fe649d25125/boto3-1.29.6-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/c9/fa/708f561f2ec441b9e9f6e9780a0519a7922a8e1f02c92671454983ff44c3/awscli-1.30.6-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/e4/dd/a6c7154d4f045599ce2df9199c01a8c2650c45ae74ad63f69547a500cc1c/botocore-1.32.6-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 81.1 MB/s eta 0:00:00
  Ob

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [5]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

## SQL 쿼리 생성

위에서 설명한 활용 사례에 따라, SQL 쿼리를 생성하기 위한 아마존 베드록 서비스의 입력을 준비해 보겠습니다.

In [6]:
# create the prompt to generate SQL query
prompt_data = """

Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- Identify the top 5 best selling products by total sales for the year 2023
- Calculate the monthly average sales for the year 2023

Assistant:
"""


먼저 Anthropic의 Claude v2 모델을 사용해 보겠습니다.

In [7]:
# Claude - Body Syntex
body = json.dumps({
                    "prompt": prompt_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

#### 베드록의 Claude 대규모 언어 모델 호출하기

우선, 앞서 작성한 프롬프트를 기반으로 모델이 어떻게 출력을 생성하는지 살펴보겠습니다.

##### 전체 출력 생성

In [9]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here are the SQL queries to answer the questions:

1. Identify the top 5 best selling products by total sales for the year 2023:

```sql
SELECT product_id, SUM(price * units_sold) AS total_sales
FROM sales_data
WHERE date BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 5;
```

2. Calculate the monthly average sales for the year 2023:

```sql 
SELECT 
  DATE_FORMAT(date, '%Y-%m') AS month,
  AVG(price * units_sold) AS avg_monthly_sales
FROM sales_data
WHERE date BETWEEN '2023-01-01' AND '2023-12-31'  
GROUP BY month
ORDER BY month;
```

The first query groups the sales data by product_id and sums the total sales for each product in 2023. It then orders by the total_sales column descending and limits to the top 5 results. 

The second query extracts the month from the date column, groups the data by month, calculates the average monthly sales using AVG() and orders the results chronologically.


### 고급 예시
#### SQL을 활용한 병원 환자 관리 시스템 분석하기

In [10]:
# create the prompt
prompt_sql_data = """

Human: You're provided with a database schema representing any hospital's patient management system.
The system holds records about patients, their prescriptions, doctors, and the medications prescribed.

Here's the schema:

```sql
CREATE TABLE Patients (
    PatientID int,
    FirstName varchar(50),
    LastName varchar(50),
    DateOfBirth datetime,
    Gender varchar(10),
    PRIMARY KEY (PatientID)
);

CREATE TABLE Doctors (
    DoctorID int,
    FirstName varchar(50),
    LastName varchar(50),
    Specialization varchar(50),
    PRIMARY KEY (DoctorID)
);

CREATE TABLE Prescriptions (
    PrescriptionID int,
    PatientID int,
    DoctorID int,
    DateIssued datetime,
    PRIMARY KEY (PrescriptionID)
);

CREATE TABLE Medications (
    MedicationID int,
    MedicationName varchar(50),
    Dosage varchar(50),
    PRIMARY KEY (MedicationID)
);

CREATE TABLE PrescriptionDetails (
    PrescriptionDetailID int,
    PrescriptionID int,
    MedicationID int,
    Quantity int,
    PRIMARY KEY (PrescriptionDetailID)
);
```

Write a SQL query that fetches all the patients who were prescribed more than 5 different medications on 2023-04-01.

Assistant:
"""


In [11]:
# Claude - Body Syntex
body = json.dumps({
                    "prompt": prompt_sql_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [12]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a SQL query to fetch patients who were prescribed more than 5 different medications on 2023-04-01:

```sql
SELECT p.FirstName, p.LastName
FROM Patients p
JOIN Prescriptions pre ON p.PatientID = pre.PatientID
JOIN PrescriptionDetails pd ON pre.PrescriptionID = pd.PrescriptionID
WHERE pre.DateIssued = '2023-04-01'
GROUP BY p.PatientID
HAVING COUNT(DISTINCT pd.MedicationID) > 5;
```

The key steps are:

1. Join the Patients, Prescriptions and PrescriptionDetails tables to connect patients with their prescriptions and medication details. 

2. Filter to only prescriptions issued on 2023-04-01.

3. Group by PatientID and count the distinct MedicationIDs per patient. 

4. Use HAVING clause to only keep patients with more than 5 distinct medications.

5. Select the first and last name columns for the matching patients.
